In [1]:
import pandas as pd
import numpy as np

#需要优化的量

#关系矩阵
#ctd
M1=np.zeros((2,18)) #国产
M3=np.zeros((2,18)) #进口
#dts
M2=np.zeros((20,106)) #国产
M4=np.zeros((20,106)) #进口

#仓库状态矩阵
O=np.zeros((20,1))  #是否开仓
Z=np.zeros((20,1))  #是否智能


In [10]:
#参数导入
#国产，对应1
Q_dm=pd.read_csv('data_p/Q_dm.csv')
Q_dm=Q_dm.to_numpy()
Q_dm=Q_dm[:,1]
Q_dm

array([ 37.6,  57.5,  19.1, 363.4,  23.3,  33. ,  77. , 104.9,  45.1,
        15.8, 157.1, 142.9,  22.7,  74.4,  25.1,  24.6, 103.3,  21.8,
        70. ,  37.5,  81.3,  33.2,  50.3, 160.7,  42.1, 107.9,  37.7,
       157.3, 126.7,  33.7,  23.6,  24. ,  27.8,  28.2,  47. ,  23.3,
        83.5,  41.6,  49.2,  21.3,  30. ,  68.7,  23.2,  56.5,  20.7,
        36.6,  25.2,  22.4,  56. ,  19.8,  31.7,  24.1,  47.2, 117.1,
        49.3,  51.8,  25.3,  53.6,  79.2,  23.4, 104.3,  73.2,  22. ,
        38.3, 246. ,  37.3,  26.4,  42.8,  95.3, 166.1,  90.7,  30.5,
        27.9,  95.8,  22.6,  82.9,  50.5,  26. ,  36.1, 142.6,  64.4,
        25.4, 106.5, 112.2,  26.8,  71.6, 155. ,  22. ,  21.3,  40.4,
        26.4,  25.5,  66.6,  49.7,  58.7,  25.2,  32.3,  25.6,  22.1,
        32.3,  26.2, 118.3,  33.2,  30.5,  21.9,  26.8])

In [11]:
#港口，对应0
Q_im=pd.read_csv('data_p/Q_im.csv')
Q_im=Q_im.to_numpy()
Q_im=Q_im[:,1]
Q_im


array([ 15.3,  23.6,  11.1, 328.6,   7.7,  16.3,  45.8,  48.8,  35.9,
        15. , 145. ,  91.5,  17.1,  49.5,  11.5,   7.9,  41.8,  10.6,
        35.6,   9.6,  45.7,  11.1,  30.2, 108.9,  29.7,  65.4,  12.7,
       586.6,  70.9,   9.8,   7.3,  24.7,  15.7,  20.3,  17.5,   9. ,
        58. ,  15.8,  28.3,  10.5,  11.6,  45.3,   8.4,  31.4,  19.6,
        16.3,   7.9,   8.3,  22.5,  12.7,  12.8,   9. ,  23.8, 101.5,
        25.9,  50.6,   7.6,  29. ,  57.6,   9. ,  75.5,  35. ,   8.2,
        20. , 449.2,   9.6,   9.5,  46.9,  61. , 109.7,  33. ,   8.7,
         7.7,  69.7,   8.7,  39.9,  23.9,  14.6,  21.1,  87. ,  24.3,
        13. ,  59.9,  93.1,  13.3,  43.9,  85.4,   8.3,  12.1,  12.4,
         8.5,   7.8,  28.9,  18.4,  46.6,  11.8,   8.7,   8.4,   8.3,
        12.8,  13.4,  57.3,  13.9,   7.9,  13. ,  13.5])

In [15]:
L_dts=pd.read_csv('data_p/d_matrix_dts.csv')
L_dts=L_dts.to_numpy()
L_dts=L_dts[1:]
L_dts=L_dts[:,2:]
L_dts

array([[671.0, 159.0, 662.0, ..., 786.0, 2262.0, 418.0],
       [2116.0, 1343.0, 1861.0, ..., 724.0, 812.0, 1290.0],
       [2506.0, 1712.0, 1609.0, ..., 1335.0, 1686.0, 1769.0],
       ...,
       [1785.0, 1023.0, 1674.0, ..., 401.0, 1114.0, 957.0],
       [1735.0, 942.0, 835.0, ..., 671.0, 1817.0, 1022.0],
       [1332.0, 539.0, 1081.0, ..., 203.0, 1602.0, 551.0]], dtype=object)

In [19]:
T_dts=pd.read_csv('data_p/t_matrix_dts.csv')
T_dts=T_dts.to_numpy()
T_dts=T_dts[1:]
T_dts=T_dts[:,2:]
T_dts

array([[532.0, 167.0, 491.0, ..., 578.0, 1478.0, 364.0],
       [1376.0, 959.0, 1295.0, ..., 556.0, 644.0, 935.0],
       [1639.0, 1209.0, 1163.0, ..., 962.0, 1130.0, 1253.0],
       ...,
       [1169.0, 758.0, 1140.0, ..., 310.0, 877.0, 709.0],
       [1189.0, 753.0, 651.0, ..., 501.0, 1249.0, 816.0],
       [948.0, 406.0, 859.0, ..., 175.0, 1137.0, 440.0]], dtype=object)

In [23]:
L_ctd=pd.read_csv('data_p/d_matrix_ctd.csv')
L_ctd=L_ctd.to_numpy()
L_ctd=L_ctd[:,2:]
L_ctd

array([[2071.0, 889.0, 2055.0, 1753.0, 2805.0, 598.0, 648.0, 3311.0,
        934.0, 1160.0, 834.0, 1694.0, 1028.0, 577.0, 1997.0, 1031.0,
        1820.0, 1532.0],
       [57.0, 1461.0, 1841.0, 1752.0, 837.0, 2131.0, 2122.0, 1346.0,
        1276.0, 999.0, 1327.0, 651.0, 1154.0, 2147.0, 91.0, 1132.0,
        1071.0, 698.0]], dtype=object)

In [24]:
T_ctd=pd.read_csv('data_p/t_matrix_ctd.csv')
T_ctd=T_ctd.to_numpy()
T_ctd=T_ctd[:,2:]
T_ctd

array([[1332.0, 649.0, 1349.0, 1209.0, 1732.0, 421.0, 449.0, 2024.0,
        696.0, 852.0, 624.0, 1148.0, 801.0, 404.0, 1298.0, 757.0, 1212.0,
        1032.0],
       [110.0, 1026.0, 1279.0, 1247.0, 682.0, 1386.0, 1391.0, 1003.0,
        949.0, 738.0, 976.0, 491.0, 885.0, 1398.0, 115.0, 832.0, 813.0,
        496.0]], dtype=object)

In [33]:
Pf=pd.read_csv('data_p/Pf.csv')
Pf=Pf.to_numpy()
Pf=Pf[:,1:]
Pf

array([[0.0073  , 0.00365 ],
       [0.00635 , 0.003175],
       [0.0067  , 0.00335 ],
       [0.0068  , 0.0034  ],
       [0.007   , 0.0035  ],
       [0.007   , 0.0035  ],
       [0.0075  , 0.00375 ],
       [0.0075  , 0.00375 ],
       [0.0075  , 0.00375 ],
       [0.0072  , 0.0036  ],
       [0.007   , 0.0035  ],
       [0.0065  , 0.00325 ],
       [0.0075  , 0.00375 ],
       [0.007   , 0.0035  ],
       [0.0072  , 0.0036  ],
       [0.0075  , 0.00375 ],
       [0.0068  , 0.0034  ],
       [0.0069  , 0.00345 ],
       [0.0065  , 0.00325 ],
       [0.007   , 0.0035  ]])

In [38]:
Of=pd.read_csv('data_p/Of.csv')
Of=Of.to_numpy()
Of=Of[:,1]
Of

array([25., 25., 25., 25., 25., 25.,  0., 25., 25., 25., 25.,  0., 25.,
       25., 25., 25., 25., 25., 25., 25.])

In [41]:
Cap=pd.read_csv('data_p/Cap.csv')
Cap=Cap.to_numpy()
Cap=Cap[:,1]
Cap

array([ 1200.,  1200.,  2000.,  2200.,  1140.,  1300.,  8000.,  2500.,
        1050.,  1500.,  2500., 10000.,  2500.,  1000.,  3500.,  1800.,
        3000.,   800.,  1200.,  1450.])

In [ ]:
#约束条件
##时间
Q_all=Q_im.sum(axis=0).sum(axis=1)
def time_lim():
    Q_out=0
    tmp1=M1*T_dts
    tmp2=M3*T_dts
    print(tmp1,"\n",tmp2)
    for i in (0,106):
        if tmp1.sum(axis=1)>=10:
            Q_out+=Q_im[i]
        if tmp2.sum(axis=1)>=10:
            Q_out+=Q_dm[i]
    out_point=Q_out/Q_all
    if out_point<=0.95:
        return True
    return False

#最大订单量不超过最大承载量
def cap_lim():
    tmp=M1.T*Q_im+M3.T*Q_dm
    tmp1=tmp.sum(axis=1)
    for i,j in tmp1,Cap:
        if i>j:
            return False
    return True


In [ ]:
#目标优化式
##处置成本
def ccl_cp():
    for i in (0,20):
